In [1]:
import scipy.constants as const
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
import os
import config

In [2]:
userHome = config.userHome()
os.chdir(userHome + '/projects/cs/methods/kosarim/')

In [3]:
pi = const.pi
e = const.e
e0= const.epsilon_0

In [4]:
eV_per_J = const.physical_constants["joule-electron volt relationship"][0]
cm_per_m = 100.0

In [8]:
"Note this equation is not used here."
def f_eqn6(x: float):
    "Equation 6 of Kosarim, et al., for s electrons."
    return (10*(x-1))/(pi*x*(x+8))

In [9]:
"Note this equation is not used here."
def f_eqn7(x: float):
    "Equation 7 of Kosarim, et al., for p electrons."
    return (10*(x-1))/(pi*(x+1.5)*(x+9))

In [10]:
def f_eqn15(x: float):
    "Equation 15 of Kosarim, et al., for all electrons."
    "return (6*(x-1))/(pi*(x+1.5)*(x+9))"
    return (6*(x-1))/(pi*(x+10)*(x+2))

In [11]:
def sigma_ion(f, n: int, E: float, J_v_vp: float, J_v_vpp: float):
    """Compute cross-sections (10^-17 cm^2) of ionization transitions for Tables 2-4 of Kosarim, et al."""
    return (cm_per_m**2 * eV_per_J**2)/(1.0E-17)  * (1./(4.*pi*e0)**2) * \
        ((n*e**4)/2.) * ( f(E/J_v_vp)/(J_v_vp**2) + f(E/J_v_vpp)/(J_v_vpp**2) )

In [12]:
def calculate_kosarim_table_sigmas_unscaled(table_ref, f_eqn, n):
    sigmas = np.ones((len(table_ref),3))
    
    for i in range(len(table_ref)):
        for j in range(3):
            sigmas[i,j] = sigma_ion(f=f_eqn, n=n, E=table_ref[i,j+6], J_v_vp=table_ref[i,1], J_v_vpp=table_ref[i,2])
                
    return sigmas

In [13]:
sigmas_X=calculate_kosarim_table_sigmas_unscaled(table2_ref, f_eqn15, 2)
sigmas_A=calculate_kosarim_table_sigmas_unscaled(table3_ref, f_eqn15, 4)
sigmas_B=calculate_kosarim_table_sigmas_unscaled(table4_ref, f_eqn15, 2)

In [14]:
sigma_tot = np.array([1.74, 6.10, 10.90])

In [15]:
N_20eV = 1.74/(sigmas_X[0,0]+sigmas_A[0,0]+sigmas_B[0,0])
N_30eV = 6.10/(sigmas_X[0,1]+sigmas_A[0,1]+sigmas_B[0,1])
N_50eV = 10.9/(sigmas_X[0,2]+sigmas_A[0,2]+sigmas_B[0,2])

In [16]:
np.set_printoptions(precision=2)

In [17]:
sigmas_X_scaled = np.column_stack((N_20eV*sigmas_X[...,0], N_30eV*sigmas_X[...,1], N_50eV*sigmas_X[...,2]))
sigmas_X_scaled

array([[0.82, 2.12, 3.49],
       [0.88, 2.2 , 3.59],
       [0.9 , 2.23, 3.63],
       [0.94, 2.29, 3.69],
       [0.94, 2.29, 3.7 ],
       [0.97, 2.33, 3.75],
       [1.02, 2.4 , 3.83],
       [1.03, 2.41, 3.84],
       [1.08, 2.48, 3.92],
       [1.11, 2.51, 3.96],
       [1.12, 2.53, 3.98]])

In [18]:
sigmas_A_scaled = np.column_stack((N_20eV*sigmas_A[...,0], N_30eV*sigmas_A[...,1], N_50eV*sigmas_A[...,2]))
sigmas_A_scaled

array([[0.79, 2.94, 5.33],
       [0.9 , 3.08, 5.48],
       [0.96, 3.13, 5.5 ],
       [1.05, 3.24, 5.62],
       [1.19, 3.45, 5.88],
       [1.37, 3.71, 6.2 ],
       [1.46, 3.85, 6.39],
       [1.55, 4.  , 6.58],
       [1.65, 4.16, 6.79],
       [1.76, 4.32, 7.  ],
       [1.88, 4.5 , 7.23]])

In [19]:
sigmas_B_scaled = np.column_stack((N_20eV*sigmas_B[...,0], N_30eV*sigmas_B[...,1], N_50eV*sigmas_B[...,2]))
sigmas_B_scaled

array([[0.13, 1.03, 2.08],
       [0.16, 1.09, 2.16],
       [0.2 , 1.17, 2.26],
       [0.26, 1.26, 2.38],
       [0.32, 1.36, 2.52],
       [0.39, 1.47, 2.66],
       [0.47, 1.59, 2.82],
       [0.58, 1.76, 3.03],
       [0.68, 1.91, 3.22],
       [0.77, 2.04, 3.39],
       [0.86, 2.16, 3.54]])

In [20]:
print('sigmas_X_scaled RMSE: {:3.4f}'.format(sqrt(mse(sigmas_X_scaled,table2_ref[...,3:6]))))

sigmas_X_scaled RMSE: 0.0453


In [21]:
print('sigmas_A_scaled RMSE: {:3.4f}'.format(sqrt(mse(sigmas_A_scaled,table3_ref[...,3:6]))))

sigmas_A_scaled RMSE: 0.0758


In [22]:
print('sigmas_B_scaled RMSE: {:3.4f}'.format(sqrt(mse(sigmas_B_scaled,table4_ref[...,3:6]))))

sigmas_B_scaled RMSE: 0.0554


In [23]:
table5_ref = np.loadtxt(open("kosarim_table_5.csv", "rb"), delimiter=" ", skiprows=1)
table5_ref

array([[ 0. ,  9.6, 11.6,  1.4,  3.1,  4.2, 15. , 25. , 40. ],
       [ 4. ,  8.8, 12. ,  1.7,  3.5,  4.6, 15. , 25. , 40. ],
       [ 8. ,  8.2, 12.4,  2.1,  4. ,  5.1, 15. , 25. , 40. ],
       [12. ,  7.6, 12.8,  2.6,  4.5,  5.6, 15. , 25. , 40. ],
       [16. ,  7.1, 13.2,  3. ,  5.1,  6.2, 15. , 25. , 40. ],
       [20. ,  6.8, 13.6,  3.4,  5.5,  6.6, 15. , 25. , 40. ],
       [24. ,  6.6, 14. ,  3.7,  5.8,  6.8, 15. , 25. , 40. ]])

In [24]:
sigmas_triplet=calculate_kosarim_table_sigmas_unscaled(table5_ref, f_eqn15, 2)
sigmas_triplet

array([[0.82, 1.86, 2.54],
       [1.02, 2.12, 2.81],
       [1.23, 2.37, 3.06],
       [1.52, 2.72, 3.4 ],
       [1.84, 3.09, 3.75],
       [2.08, 3.35, 3.99],
       [2.25, 3.53, 4.15]])

In [25]:
"""
Need to figure out how to scale the triplet sigmas. For now, use
the same scaling factors as above even though we know that's wrong.
"""
N_15eV = 1.74/(sigmas_X[0,0]+sigmas_A[0,0]+sigmas_B[0,0])
N_25eV = 6.10/(sigmas_X[0,1]+sigmas_A[0,1]+sigmas_B[0,1])
N_40eV = 10.9/(sigmas_X[0,2]+sigmas_A[0,2]+sigmas_B[0,2])

In [26]:
sigmas_triplet_scaled = \
    np.column_stack((N_15eV*sigmas_triplet[...,0], N_25eV*sigmas_triplet[...,1], N_40eV*sigmas_triplet[...,2]))
sigmas_triplet_scaled

array([[ 2.71,  6.14,  8.5 ],
       [ 3.37,  6.99,  9.39],
       [ 4.07,  7.83, 10.24],
       [ 5.03,  8.98, 11.37],
       [ 6.1 , 10.2 , 12.56],
       [ 6.87, 11.05, 13.34],
       [ 7.45, 11.65, 13.89]])

In [27]:
print('sigmas_triplet_scaled RMSE: {:3.4f}'.format(sqrt(mse(sigmas_triplet_scaled,table5_ref[...,3:6]))))

sigmas_triplet_scaled RMSE: 4.5494
